In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,60,2024-02-29,3,121.813559,100.508475,0.575102,11.072881,21.910169,0.220593,120.561017,...,11.915517,21.822414,0.168414,110.212069,0,2,3,2,17,2024
1,60,2024-02-29,-1,117.661017,99.498305,0.543525,13.111864,28.928814,0.212000,117.249153,...,12.500000,26.067797,0.224441,114.440678,0,2,3,2,29,2024
2,58,2024-02-29,1,119.245614,99.082456,0.556982,12.084211,27.487719,0.189509,119.103509,...,11.086441,29.555932,0.204288,119.225424,0,2,3,2,10,2024
3,59,2024-02-29,2,121.155172,100.660345,0.539776,10.855172,27.815517,0.217914,119.472414,...,10.762069,24.784483,0.175103,114.810345,0,2,3,2,1,2024
4,59,2024-02-29,6,121.293103,99.146552,0.579293,10.534483,20.648276,0.205017,121.491379,...,12.615254,22.564407,0.171797,110.467797,0,2,3,2,21,2024
5,59,2024-02-29,-2,112.827586,98.498276,0.526276,11.222414,24.515517,0.204534,113.608621,...,12.606897,24.765517,0.240948,119.462069,0,2,3,2,11,2024
6,59,2024-02-29,5,110.603448,96.081034,0.538000,11.375862,22.243103,0.221397,114.925862,...,10.712069,26.227586,0.178931,119.301724,0,2,3,2,16,2024
7,59,2024-02-29,-12,114.327586,102.750000,0.538103,11.837931,19.950000,0.173793,111.256897,...,11.801695,19.547458,0.214542,115.713559,0,2,3,2,30,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-29,MIL,CHO,1.142857,6.000000,1
1,2024-02-29,UTA,ORL,3.050000,1.400000,0
2,2024-02-29,GSW,NYK,1.540541,2.540000,0
3,2024-02-29,ATL,BRK,2.020000,1.819672,0
4,2024-02-29,OKC,SAS,1.160000,5.550000,1
5,2024-02-29,HOU,PHO,3.950000,1.266667,0
6,2024-02-29,MIA,DEN,2.640000,1.505051,0
7,2024-02-29,WAS,LAL,4.200000,1.243902,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-29,MIL,CHO,1.142857,6.000000,1
1,2024-02-29,UTA,ORL,3.050000,1.400000,0
2,2024-02-29,GSW,NYK,1.540541,2.540000,0
3,2024-02-29,ATL,BRK,2.020000,1.819672,0
4,2024-02-29,OKC,SAS,1.160000,5.550000,1
5,2024-02-29,HOU,PHO,3.950000,1.266667,0
6,2024-02-29,MIA,DEN,2.640000,1.505051,0
7,2024-02-29,WAS,LAL,4.200000,1.243902,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-29,MIL,CHO,1.142857,6.000000,1
1,2024-02-29,UTA,ORL,3.050000,1.400000,0
2,2024-02-29,GSW,NYK,1.540541,2.540000,0
3,2024-02-29,ATL,BRK,2.020000,1.819672,0
4,2024-02-29,OKC,SAS,1.160000,5.550000,1
5,2024-02-29,HOU,PHO,3.950000,1.266667,0
6,2024-02-29,MIA,DEN,2.640000,1.505051,0
7,2024-02-29,WAS,LAL,4.200000,1.243902,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-29,MIL,CHO,1.142857,6.000000,1
1,2024-02-29,UTA,ORL,3.050000,1.400000,0
2,2024-02-29,GSW,NYK,1.540541,2.540000,0
3,2024-02-29,ATL,BRK,2.020000,1.819672,0
4,2024-02-29,OKC,SAS,1.160000,5.550000,1
5,2024-02-29,HOU,PHO,3.950000,1.266667,0
6,2024-02-29,MIA,DEN,2.640000,1.505051,0
7,2024-02-29,WAS,LAL,4.200000,1.243902,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-29,MIL,CHO,1.142857,6.000000,1.0
1,2024-02-29,UTA,ORL,3.050000,1.400000,1.0
2,2024-02-29,GSW,NYK,1.540541,2.540000,1.0
3,2024-02-29,ATL,BRK,2.020000,1.819672,1.0
4,2024-02-29,OKC,SAS,1.160000,5.550000,1.0
5,2024-02-29,HOU,PHO,3.950000,1.266667,0.0
6,2024-02-29,MIA,DEN,2.640000,1.505051,0.0
7,2024-02-29,WAS,LAL,4.200000,1.243902,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-29,MIL,CHO,1.142857,6.000000,1.0
1,2024-02-29,UTA,ORL,3.050000,1.400000,1.0
2,2024-02-29,GSW,NYK,1.540541,2.540000,1.0
3,2024-02-29,ATL,BRK,2.020000,1.819672,1.0
4,2024-02-29,OKC,SAS,1.160000,5.550000,1.0
5,2024-02-29,HOU,PHO,3.950000,1.266667,0.0
6,2024-02-29,MIA,DEN,2.640000,1.505051,0.0
7,2024-02-29,WAS,LAL,4.200000,1.243902,0.0
